<a href="https://colab.research.google.com/github/magicaltrap/english_german_translation/blob/master/preprocessing_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
!pip install fasttext

In [0]:
import fasttext as ft

In [0]:
from fastai.text import *

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
path_translation = "/content/gdrive/My Drive/data/translation/"
path_translation_data = "/content/gdrive/My Drive/data/translation/data/"

# Preprocess our data

I donwloaded news-commentary-v9 and the other validation and test sets from the WMT2014 conference from this website (all preprocessed, no .sgm file like on the WMT homepage): https://google.github.io/seq2seq/data/

## Training set (news-commentary-v9)

In [0]:
en_train_fname = path_translation_data + 'news-commentary-v9.de-en.en'
de_train_fname = path_translation_data + 'news-commentary-v9.de-en.de'

In [0]:
with open(en_train_fname, encoding='utf-8') as en_file, open(de_train_fname, encoding='utf-8') as de_file:
  en_list = []
  for en in en_file:
    if en:
      en_list.append(en)
    
  de_list = []
  for de in de_file:
    if de:
      de_list.append(de)  

In [0]:
len(en_list), len(de_list)

(201995, 201854)

German and English `news-commentary-v9 files (WMT2014)` don't have the same length respectively... So let's donwload a newer version of the news-commentary corpus.

## Training set (news-commentary-v14)

Download this dataset from the WMT2019 website: http://www.statmt.org/wmt19/translation-task.html

In [0]:
v14_train_fname = path_translation_data + 'news-commentary-v14.de-en.tsv'

`news-commentary-v14` is a bit differently structured, German and English language pairs are separated by a "\t" (instead of two separate files like `news-commentary-v9`)

In [0]:
with open(v14_train_fname, encoding='utf-8') as v14_file:
  v14_lines = []
  for sent in v14_file:
    if "\t" in sent:
      try:
        de, en = sent.rstrip("\n").split("\t")
      except ValueError: #if for some reasons (false formatting etc.) there are more than one "\t" (tab) in one line so we can't separate German and English sentences cleanly
        pass
      
        #count_tab = sent.count("\t")
        #print(count_tab)
        #print(sent)
      
      v14_lines.append((en, de))

In [0]:
len(v14_lines)

338285

In [0]:
v14_lines[:5]

[('$10,000 Gold?', 'Steigt Gold auf 10.000 Dollar?'),
 ('SAN FRANCISCO – It has never been easy to have a rational conversation about the value of gold.',
  'SAN FRANCISCO – Es war noch nie leicht, ein rationales Gespräch über den Wert von Gold zu führen.'),
 ('Lately, with gold prices up more than 300% over the last decade, it is harder than ever.',
  'In letzter Zeit allerdings ist dies schwieriger denn je, ist doch der Goldpreis im letzten Jahrzehnt um über 300 Prozent angestiegen.'),
 ('Just last December, fellow economists Martin Feldstein and Nouriel Roubini each penned op-eds bravely questioning bullish market sentiment, sensibly pointing out gold’s risks.',
  'Erst letzten Dezember verfassten meine Kollegen Martin Feldstein und Nouriel Roubini Kommentare, in denen sie mutig die vorherrschende optimistische Marktstimmung hinterfragten und sehr überlegt auf die Risiken des Goldes \xa0hinwiesen.'),
 ('Wouldn’t you know it?', 'Und es kam, wie es kommen musste.')]

Save it in a dataframe:

In [0]:
v14_df_train = pd.DataFrame({'en': [sent[0] for sent in v14_lines], 'de': [sent[1] for sent in v14_lines]}, columns = ['en', 'de'])

Check if there are empty rows(lines)

In [0]:
v14_df_train.isnull().any()

en    False
de    False
dtype: bool

In [0]:
v14_df_train.iloc[:10]

,en,de
0,"$10,000 Gold?",Steigt Gold auf 10.000 Dollar?
1,SAN FRANCISCO – It has never been easy to have a rational conversation about the value of gold.,"SAN FRANCISCO – Es war noch nie leicht, ein rationales Gespräch über den Wert von Gold zu führen."
2,"Lately, with gold prices up more than 300% over the last decade, it is harder than ever.","In letzter Zeit allerdings ist dies schwieriger denn je, ist doch der Goldpreis im letzten Jahrzehnt um über 300 Prozent angestiegen."
3,"Just last December, fellow economists Martin Feldstein and Nouriel Roubini each penned op-eds bravely questioning bullish market sentiment, sensibly pointing out gold’s risks.","Erst letzten Dezember verfassten meine Kollegen Martin Feldstein und Nouriel Roubini Kommentare, in denen sie mutig die vorherrschende optimistische Marktstimmung hinterfragten und sehr überlegt auf die Risiken des Goldes hinwiesen."
4,Wouldn’t you know it?,"Und es kam, wie es kommen musste."
5,"Since their articles appeared, the price of gold has moved up still further.",Seit der Veröffentlichung ihrer Artikel ist der Goldpreis noch weiter gestiegen.
6,"Gold prices even hit a record-high $1,300 recently.",Jüngst erreichte er sogar ein Rekordhoch von 1.300 Dollar.
7,"Last December, many gold bugs were arguing that the price was inevitably headed for $2,000.","Im letzten Dezember argumentierten die Goldbugs, dass der Preis zweifellos in Richtung 2.000 Dollar gehen würde."
8,"Now, emboldened by continuing appreciation, some are suggesting that gold could be headed even higher than that.","Beflügelt aufgrund des anhaltenden Aufwärtstrends, meint man nun mancherorts, dass Gold sogar noch höher steigen könnte."
9,"One successful gold investor recently explained to me that stock prices languished for a more than a decade before the Dow Jones index crossed the 1,000 mark in the early 1980’s.","Ein erfolgreicher Gold-Investor erklärte mir vor kurzem, dass die Aktienkurse über ein Jahrzehnt dahingedümpelt waren, bevor der Dow Jones-Index in den frühen 1980er Jahren die Marke von 1.000 Punkten überschritt."


In [0]:
v14_df_train.iloc[-10:]

,en,de
338275,Zuma senses the urgency of the situation.,Zuma weiß um die Dringlichkeit der Situation.
338276,"He is, after all, 67 years old and likely to serve only a single term in office. “We can’t waste time,” he says.","Immerhin ist er 67 Jahre alt und wird wahrscheinlich nur eine Amtszeit dienen. „Wir können uns keine Zeitverschwendung leisten“, sagt er."
338277,"Yet, according to the political economist Moeletsi Mbeki, at his core, “Zuma is a conservative.” In this sense, Zuma represents yesterday’s South Africa.","Dem politischen Ökonomen Moeletsi Mbeki zufolge, ist Zuma im Grunde seines Herzens „ein Konservativer“. In diesem Sinne vertritt Zuma das Südafrika von gestern."
338278,He is part of the proud generation that defeated apartheid – and then peacefully engineered a transition to durable black-majority rule.,"Er ist Mitglied einer stolzen Generation, die die Apartheid bezwang – und der anschließend ein friedlicher Übergang zu einer schwarzen Mehrheitsregierung gelang."
338279,Their achievement remains one of the greatest in recent history.,Das bleibt eine der größten Errungenschaften in der jüngeren Geschichte.
338280,"At the same time, Zuma’s revolutionary generation still seems uneasy leading South Africa in a post-apartheid era that is now 15 years old.",Gleichzeitig scheint sich Zumas revolutionäre Generation mit der Führung Südafrikas in der nun seit 15 Jahren dauernden Ära nach der Apartheid noch immer unwohl zu fühlen.
338281,"In a region that reveres the elderly, Zuma’s attachment to his rural traditions must be matched by an equal openness to the appetites of the country’s youth.","In einer Region, wo die älteren Menschen sehr verehrt werden, muss Zumas Bindung an landestypische Traditionen eine gleichwertige Offenheit gegenüber den Bedürfnissen der Jugend des Landes gegenüberstehen."
338282,"Three in ten South Africans are younger than 15, meaning that they did not live a day under apartheid.","Drei von zehn Südafrikanern sind jünger als 15 und das bedeutet, dass sie nicht einen Tag unter der Apartheid gelebt haben."
338283,"Somehow Zuma must find a way to honor his own generation’s commitment to racial justice and national liberation, while empowering the masses who daily suffer the sting of class differences and yearn for material gain.","Irgendwie muss Zuma einen Weg finden, einerseits das Engagement seiner Generation hinsichtlich ethnischer Gerechtigkeit und nationaler Befreiung zu würdigen und andererseits den Massen, die täglich unter Klassenunterschieden leiden und sich nach materiellen Verbesserungen sehnen, mehr Mitwirkung..."
338284,,


In [0]:
v14_df_train = v14_df_train.drop([338284]) #drop empty row

Save dataframe as csv file:

In [0]:
v14_df_train.to_csv(path_translation_data + 'news-commentary-v14.en-de.csv', index=False)

In [0]:
v14_df_train_load = pd.read_csv(path_translation_data + 'news-commentary-v14.en-de.csv')

Dataframe before we saved into a csv file:

In [0]:
v14_df_train.isnull().any() #old dataframe

en    False
de    False
dtype: bool

In [0]:
v14_df_train.isnull().sum()

en    0
de    0
dtype: int64

New dataframe that was loaded from the saved csv file: Now 8799 English and 9780 German empty (isnull()) rows are detected which weren't detected before:

In [0]:
v14_df_train_load.isnull().any() #new (loaded) dataframe

en    True
de    True
dtype: bool

In [0]:
v14_df_train_load.isnull().sum()

en    8799
de    8780
dtype: int64

### Remove empty rows from news-commentary-v14

In [0]:
null_rows = v14_df_train_load[v14_df_train_load["en"].isnull()]

In [0]:
null_en_rows_indices = null_rows.index.values
null_rows_indices

array([    46,     77,    117,    127, ..., 338155, 338183, 338214, 338252])

In the old dataframe, the empty rows were not recognized:

In [0]:
v14_df_train.iloc[46]

en    
de    
Name: 46, dtype: object

In contrast to the new dataframe:

In [0]:
v14_df_train_load.iloc[46]

en    NaN
de    NaN
Name: 46, dtype: object

remove empty lines:

In [0]:
null_en_rows = v14_df_train_load[v14_df_train_load["en"].isnull()].index.values
null_de_rows = v14_df_train_load[v14_df_train_load["de"].isnull()].index.values

In [0]:
v14_null_rows = np.unique(np.concatenate([null_en_rows, null_de_rows]))
v14_null_rows

array([    46,     77,    117,    127, ..., 338155, 338183, 338214, 338252])

In [0]:
v14_df_train_final = v14_df_train_load.drop(list(v14_null_rows))

No empty rows anymore:

In [0]:
v14_df_train_final.isnull().any()

en    False
de    False
dtype: bool

In [0]:
v14_df_train_final.to_csv(path_translation_data + 'news-commentary-v14.en-de.csv', index=False)

In [0]:
v14_df_train_final = pd.read_csv(path_translation_data + 'news-commentary-v14.en-de.csv')

### Smaller subset

Let's create a smaller subset (from 329127 languages pairs to 50000) just in case (faster training, testing etc.)

In [0]:
len(v14_df_train_final)

329167

In [0]:
new_v14 = v14_df_train_final.copy()
v14_50k = new_v14.sample(50000).reset_index(drop=True)
len(v14_50k)

50000

In [0]:
v14_50k.isnull().any()

en    False
de    False
dtype: bool

In [0]:
v14_50k.head()

,en,de
0,"In China, people call it fazhan , or “development,” but in much of rest of the world, it is more commonly described simply as the “China Boom,” or the “China Miracle.”","In China sagt man dazu fazhan , also „Entwicklung”, aber in den meisten Teilen der Welt wird dieses Phänomen einfach als „China-Boom“ oder „chinesisches Wunder“ bezeichnet."
1,Such pessimism is warranted.,Ein derartiger Pessimismus ist gerechtfertigt.
2,"That is also what, if you take her literally, Merkel is doing now.","Dasselbe tut derzeit auch Merkel, wenn man sie beim Wort nimmt."
3,"It proved to be the tipping point, triggering waves of protests by lawyers and other groups in Pakistan’s main cities.","Der Vorfall erwies sich als Wendepunkt, der Wellen des Protests von Anwälten und anderen Gruppen in Pakistans größten Städten auslöste."
4,"Welfare-to-work programs in Michigan and Wisconsin served as the model for federal welfare reform under President Bill Clinton, and Obamacare is based on Massachusetts’ health-care system, introduced under Republican Governor Mitt Romney.","Arbeitsbeschaffungsprogramme in Michigan und Wisconsin dienten als Modell für die staatliche Sozialreform unter Präsident Bill Clinton, und das Krankenversicherungsprogramm „Obamacare“ baut auf dem Gesundheitssystem von Massachusetts auf, das unter dem republikanischen Gouverneur Mitt Romney eingeführt wurde."


In [0]:
v14_50k.to_csv(path_translation_data + 'news-commentary-50k-v14.en-de.csv', index=False)

## Validation set (newstest2013)

For validation and test sets, we have two options. Either 1) we take a small subset from our training set and label them as validation and/or test set or 2) we download a separate validation and/or test set from different sources. Let's do both just in case. For option 2) let's download the `newstest2013` and  `newstest2014` corpora as validation and test sets respectively. Those two corpora are popular in Machine Translation research to validate MT models. The original Transformer model (["Attention Is All You Need"](https://arxiv.org/abs/1706.03762)) used those two corpora as well for example. 

In [0]:
en_valid_fname = path_translation_data + 'newstest2013.en'
de_valid_fname = path_translation_data + 'newstest2013.de'

In [0]:
with open(en_valid_fname, encoding='utf-8') as en_file, open(de_valid_fname, encoding='utf-8') as de_file:
  en_list = []
  for en in en_file:
    en_list.append(en)
    
  de_list = []
  for de in de_file:
    de_list.append(de) 
    
len(en_list), len(de_list)

(3000, 3000)

In [0]:
lines_valid = [(en.rstrip(), de.rstrip()) for en, de in zip(open(en_valid_fname, encoding='utf-8'), open(de_valid_fname, encoding='utf-8'))]

In [0]:
len(lines_valid)

3000

In [0]:
lines_valid[:5]

[('A Republican strategy to counter the re-election of Obama',
  'Eine republikanische Strategie, um der Wiederwahl von Obama entgegenzutreten'),
 ('Republican leaders justified their policy by the need to combat electoral fraud.',
  'Die Führungskräfte der Republikaner rechtfertigen ihre Politik mit der Notwendigkeit, den Wahlbetrug zu bekämpfen.'),
 ('However, the Brennan Centre considers this a myth, stating that electoral fraud is rarer in the United States than the number of people killed by lightning.',
  'Allerdings hält das Brennan Center letzteres für einen Mythos, indem es bekräftigt, dass der Wahlbetrug in den USA seltener ist als die Anzahl der vom Blitzschlag getöteten Menschen.'),
 ('Indeed, Republican lawyers identified only 300 cases of electoral fraud in the United States in a decade.',
  'Die Rechtsanwälte der Republikaner haben in 10 Jahren in den USA übrigens nur 300 Fälle von Wahlbetrug verzeichnet.'),
 ('One thing is certain: these new provisions will have a negat

In [0]:
df_valid = pd.DataFrame({'en': [sent[0] for sent in lines_valid], 'de': [sent[1] for sent in lines_valid]}, columns = ['en', 'de'])

In [0]:
df_valid.isnull().any()

en    False
de    False
dtype: bool

In [0]:
df_valid.iloc[0:10]

,en,de
0,A Republican strategy to counter the re-election of Obama,"Eine republikanische Strategie, um der Wiederwahl von Obama entgegenzutreten"
1,Republican leaders justified their policy by the need to combat electoral fraud.,"Die Führungskräfte der Republikaner rechtfertigen ihre Politik mit der Notwendigkeit, den Wahlbetrug zu bekämpfen."
2,"However, the Brennan Centre considers this a myth, stating that electoral fraud is rarer in the United States than the number of people killed by lightning.","Allerdings hält das Brennan Center letzteres für einen Mythos, indem es bekräftigt, dass der Wahlbetrug in den USA seltener ist als die Anzahl der vom Blitzschlag getöteten Menschen."
3,"Indeed, Republican lawyers identified only 300 cases of electoral fraud in the United States in a decade.",Die Rechtsanwälte der Republikaner haben in 10 Jahren in den USA übrigens nur 300 Fälle von Wahlbetrug verzeichnet.
4,One thing is certain: these new provisions will have a negative impact on voter turn-out.,Eins ist sicher: diese neuen Bestimmungen werden sich negativ auf die Wahlbeteiligung auswirken.
5,"In this sense, the measures will partially undermine the American democratic system.",In diesem Sinne untergraben diese Maßnahmen teilweise das demokratische System der USA.
6,"Unlike in Canada, the American States are responsible for the organisation of federal elections in the United States.",Im Gegensatz zu Kanada sind die US-Bundesstaaten für die Durchführung der Wahlen in den einzelnen Staaten verantwortlich.
7,It is in this spirit that a majority of American governments have passed new laws since 2009 making the registration or voting process more difficult.,"In diesem Sinne hat die Mehrheit der amerikanischen Regierungen seit 2009 neue Gesetze verkündet, die das Verfahren für die Registrierung oder den Urnengang erschweren."
8,"This phenomenon gained momentum following the November 2010 elections, which saw 675 new Republican representatives added in 26 States.","Dieses Phänomen hat nach den Wahlen vom November 2010 an Bedeutung gewonnen, bei denen 675 neue republikanische Vertreter in 26 Staaten verzeichnet werden konnten."
9,"As a result, 180 bills restricting the exercise of the right to vote in 41 States were introduced in 2011 alone.","Infolgedessen wurden 180 Gesetzesentwürfe allein im Jahr 2011 eingeführt, die die Ausübung des Wahlrechts in 41 Staaten einschränken."


In [0]:
df_valid.iloc[-10:]

,en,de
2990,The Internet has caused a boom in these speculations.,Das Internet hat diese Spekulationen erhöht.
2991,"""In other places, people don't think about it"" he said.","""Anderenorts denken die Leute nicht so"", sagt er."
2992,It's mostly in the English-speaking world.,Dies ist hauptsächlich in der englischsprechenden Welt der Fall.
2993,"Joseph Dougherty, a Professor of religion at La Salle University, who is giving courses in the Philippines this year, responded quickly to the question of whether he knew about any courses on the ""end of the world"" there.","Joseph Dougherty, ein Religionsprofessor der Universität La Salle, der dieses Jahr auf den Philippinen unterrichtet, antwortet schnell auf die Frage, ob er dort einen Kurs über das ""Ende der Welt"" kennen würde."
2994,"""The Philippines are not taking part in the end of the world"" he wrote, suggesting an exception of a higher authority.","""Die Philippinen beteiligen sich nicht am Ende der Welt"", schrieb er als Andeutung auf eine Ausnahme einer höheren Instanz."
2995,We have an indulgence from the Pope.,Wir haben eine Begnadigung vom Papst.
2996,"Restall noted that over the years there has been talk of many days of the last judgement, and said that if nothing happens on December 21st, ""people will immediately start thinking of the next date"" or philosophising that December 21st is the beginning of a seven-year period after which the worl...","Restall wies darauf hin, dass über die Jahre viele Tage vom letzten Gericht gesprochen wurde, und sagte, dass wenn am 21. Dezember nichts passiert, ""die Leute sofort an ein neues Datum denken"" oder darüber philosophierten, dass der 21. Dezember der Beginn einer siebenjährigen Periode sei, wonach..."
2997,Students and teachers are taking the date lightly.,Studenten und Professoren nehmen dieses Datum leicht.
2998,"Some said they plan to go to ""end of the world"" parties.","Mehrere sagten, dass sie zu Feiern über ""das Ende der Welt"" gehen möchten."
2999,"""Maybe I'll call some friends so we can have a laugh together"" said Samira Ford, 20-year-old communications student.","""Vielleicht rufe ich ein paar Freunde an, damit wir zusammen lachen können"", sagte Samira Ford, eine 20-jährige Studentin für Kommunikationen."


In [0]:
df_valid.to_csv(path_translation_data + 'newstest2013.en-de.csv', index=False)

In [0]:
df_valid = pd.read_csv(path_translation_data + 'newstest2013.en-de.csv')

In [0]:
df_valid.isnull().any()

en    False
de    False
dtype: bool

## Test set (newstest2014)

In [0]:
en_test_fname = path_translation_data + 'newstest2014.en'
de_test_fname = path_translation_data + 'newstest2014.de'

In [0]:
with open(en_test_fname, encoding='utf-8') as en_file, open(de_test_fname, encoding='utf-8') as de_file:
  en_list = []
  for en in en_file:
    en_list.append(en)
    
  de_list = []
  for de in de_file:
    de_list.append(de) 
    
len(en_list), len(de_list)

(3003, 3003)

In [0]:
lines_test = [(en.rstrip(), de.rstrip()) for en, de in zip(open(en_test_fname, encoding='utf-8'), open(de_test_fname, encoding='utf-8'))]

In [0]:
len(lines_test)

3003

In [0]:
lines_test[:5]

[('Gutach: Increased safety for pedestrians',
  'Gutach: Noch mehr Sicherheit für Fußgänger'),
 ('They are not even 100 metres apart: On Tuesday, the new B 33 pedestrian lights in Dorfparkplatz in Gutach became operational - within view of the existing Town Hall traffic lights.',
  'Sie stehen keine 100 Meter voneinander entfernt: Am Dienstag ist in Gutach die neue B 33-Fußgängerampel am Dorfparkplatz in Betrieb genommen worden - in Sichtweite der älteren Rathausampel.'),
 ('Two sets of lights so close to one another: intentional or just a silly error?',
  'Zwei Anlagen so nah beieinander: Absicht oder Schildbürgerstreich?'),
 ("Yesterday, Gutacht's Mayor gave a clear answer to this question.",
  'Diese Frage hat Gutachs Bürgermeister gestern klar beantwortet.'),
 ('"At the time, the Town Hall traffic lights were installed because this was a school route," explained Eckert yesterday.',
  '"Die Rathausampel ist damals installiert worden, weil diese den Schulweg sichert", erläuterte Ecke

In [0]:
df_test = pd.DataFrame({'en': [sent[0] for sent in lines_test], 'de': [sent[1] for sent in lines_test]}, columns = ['en', 'de'])

In [0]:
df_test.isnull().any()

en    False
de    False
dtype: bool

In [0]:
df_test.iloc[:10]

,en,de
0,Gutach: Increased safety for pedestrians,Gutach: Noch mehr Sicherheit für Fußgänger
1,"They are not even 100 metres apart: On Tuesday, the new B 33 pedestrian lights in Dorfparkplatz in Gutach became operational - within view of the existing Town Hall traffic lights.",Sie stehen keine 100 Meter voneinander entfernt: Am Dienstag ist in Gutach die neue B 33-Fußgängerampel am Dorfparkplatz in Betrieb genommen worden - in Sichtweite der älteren Rathausampel.
2,Two sets of lights so close to one another: intentional or just a silly error?,Zwei Anlagen so nah beieinander: Absicht oder Schildbürgerstreich?
3,"Yesterday, Gutacht's Mayor gave a clear answer to this question.",Diese Frage hat Gutachs Bürgermeister gestern klar beantwortet.
4,"""At the time, the Town Hall traffic lights were installed because this was a school route,"" explained Eckert yesterday.","""Die Rathausampel ist damals installiert worden, weil diese den Schulweg sichert"", erläuterte Eckert gestern."
5,"The Kluser lights protect cyclists, as well as those travelling by bus and the residents of Bergle.",Die Kluser-Ampel sichere sowohl Radfahrer als auch Busfahrgäste und die Bergle-Bewohner.
6,"The system, which officially became operational yesterday, is of importance to the Sulzbachweg/Kirchstrasse junction.",Die gestern offiziell in Betrieb genommene Anlage sei wichtig für den Kreuzungsbereich Sulzbachweg/Kirchstraße.
7,"We have the museum, two churches, the spa gardens, the bus stop, a doctor's practice and a bank, not to mention the traffic from the 'Grub' residential area.","Wir haben das Museum, zwei Kirchen, Kurpark, die Bushaltestelle, einen Arzt und eine Bank sowie den Verkehrsfluss aus dem Wohngebiet ›Grub‹."
8,"""At times of high road and pedestrian traffic, an additional set of lights were required to ensure safety,"" said Eckert.","""Bei dem hohen Verkehrs- und Fußgängeraufkommen musste zu deren Sicherheit eine weitere Ampel her"", so Eckert."
9,This was also confirmed by Peter Arnold from the Offenburg District Office.,Dies bestätigt auch Peter Arnold vom Landratsamt Offenburg.


In [0]:
df_test.iloc[-10:]

,en,de
2993,Phuket police interviewed Bamford for two days before she confessed to fabricating the story.,"Die Polizei in Phuket verhörte Bamford zwei Tage lang, bis sie gestand, sich die Geschichte ausgedacht zu haben."
2994,She was held in local police cells before the court hearing.,Bis zur Gerichtsverhandlung befand sie sich vor Ort in Polizeigewahrsam.
2995,Bamford was sentenced to serve the 15-day prison term at a low security detention centre on the outskirts of Phuket rather than in an adult women's jail.,"Bamford wurde zu einer 15-tägigen Gefängnisstrafe in einer Haftanstalt mit niedriger Sicherheitsstufe am Rande Phukets verurteilt, statt in einem Gefängnis für erwachsene Frauen."
2996,"She is the daughter of former Australian league player Peter Tunks, who has appealed to the Department of Foreign Affairs in Canberra to assist his daughter.","Sie ist die Tochter von Peter Tunks, einem ehemaligen Spieler der australischen Rubgy-Liga, der sich an das Außenministerium in Canberra mit der Bitte um Hilfe für seine Tochter gewandt hat."
2997,"Tunks told Sydney's Sunday Telegraph the whole family was ""extremely concerned"" about his daughter's welfare and wanted her back in Australia.","Tunks erklärte gegenüber dem Sunday Telegraph in Sydney, die ganze Familie sei „extrem besorgt“ über das Wohlergehen seiner Tochter und wollte sie zurück in Australien haben."
2998,"""It's obviously been a worrying time but we're hopeful to have her back home safely as soon as possible,"" Tunks said.","„Wir machen uns natürlich große Sorgen, aber wir hoffen, dass sie so bald wie möglich wieder zu Hause ist“, sagte Tunks."
2999,"Bamford is appealing the sentence and has been granted bail of 50,000 baht.",Bamford hat gegen das Urteil Berufung eingelegt und ist gegen eine Kaution von 50.000 Baht auf freiem Fuß.
3000,"Reports in Australia said that in the meantime, she was holidaying at the resort area of Krabi in Southern Thailand.","In australischen Berichten war zu lesen, dass sie in der Zwischenzeit im Ferienort Krabi in Südthailand Urlaub macht."
3001,Thai-based legal sources said Bamford was being represented by a local lawyer in Phuket but warned that the appeal may lead to the court increasing her sentence by up to two years and forcing her to serve it in an adult prison.,"Vonseiten des Gerichts war zu hören, Bamford werde durch einen lokalen Anwalt in Phuket vertreten und sei gewarnt worden, dass die Berufung auch zur Verhängung einer höheren Strafe von bis zu zwei Jahren in einem Gefängnis für Erwachsene führen könne."
3002,"However, following the recent murder of Australian travel agent Michelle Smith in Phuket, Thailand may also be looking to repair its battered tourist image, leading to an acquittal.","Allerdings ist es auch möglich, dass Thailand nach der kürzlichen Ermordung der Reisekauffrau Michelle Smith in Phuket bemüht sein könnte, sein angeschlagenes touristisches Image zu verbessern, und es deshalb zu einem Freispruch kommt."


In [0]:
df_test.to_csv(path_translation_data + 'newstest2014.en-de.csv', index=False)

In [0]:
df_test = pd.read_csv(path_translation_data + 'newstest2014.en-de.csv')

In [0]:
df_test.isnull().any()

en    False
de    False
dtype: bool

# Create DataBunch

Load all csv files:

In [0]:
df_train = pd.read_csv(path_translation_data + 'news-commentary-v14.en-de.csv')
df_train_50k = pd.read_csv(path_translation_data + 'news-commentary-50k-v14.en-de.csv')
df_valid = pd.read_csv(path_translation_data + 'newstest2013.en-de.csv')
df_test = pd.read_csv(path_translation_data + 'newstest2014.en-de.csv')

In [0]:
df_train.isnull().any().any(), df_train_50k.isnull().any().any(), df_valid.isnull().any().any(), df_test.isnull().any().any()

(False, False, False, False)

Lowercase everything

In [0]:
df_train['en'] = df_train['en'].apply(lambda x:x.lower())
df_train['de'] = df_train['de'].apply(lambda x:x.lower())

df_train_50k['en'] = df_train_50k['en'].apply(lambda x:x.lower())
df_train_50k['de'] = df_train_50k['de'].apply(lambda x:x.lower())

df_valid['en'] = df_valid['en'].apply(lambda x:x.lower())
df_valid['de'] = df_valid['de'].apply(lambda x:x.lower())

df_test['en'] = df_test['en'].apply(lambda x:x.lower())
df_test['de'] = df_test['de'].apply(lambda x:x.lower())

In [0]:
pd.set_option('max_colwidth', 800)

In [0]:
df_train.tail()

,en,de
329162,their achievement remains one of the greatest in recent history.,das bleibt eine der größten errungenschaften in der jüngeren geschichte.
329163,"at the same time, zuma’s revolutionary generation still seems uneasy leading south africa in a post-apartheid era that is now 15 years old.",gleichzeitig scheint sich zumas revolutionäre generation mit der führung südafrikas in der nun seit 15 jahren dauernden ära nach der apartheid noch immer unwohl zu fühlen.
329164,"in a region that reveres the elderly, zuma’s attachment to his rural traditions must be matched by an equal openness to the appetites of the country’s youth.","in einer region, wo die älteren menschen sehr verehrt werden, muss zumas bindung an landestypische traditionen eine gleichwertige offenheit gegenüber den bedürfnissen der jugend des landes gegenüberstehen."
329165,"three in ten south africans are younger than 15, meaning that they did not live a day under apartheid.","drei von zehn südafrikanern sind jünger als 15 und das bedeutet, dass sie nicht einen tag unter der apartheid gelebt haben."
329166,"somehow zuma must find a way to honor his own generation’s commitment to racial justice and national liberation, while empowering the masses who daily suffer the sting of class differences and yearn for material gain.","irgendwie muss zuma einen weg finden, einerseits das engagement seiner generation hinsichtlich ethnischer gerechtigkeit und nationaler befreiung zu würdigen und andererseits den massen, die täglich unter klassenunterschieden leiden und sich nach materiellen verbesserungen sehnen, mehr mitwirkungsmöglichkeiten einzuräumen."


## Combine train and validation set

**split_from_df()**

If you want to use the separate `newstest2013`  or `newstest2014` as validation and test sets respectively, you can concatenate them with the training set in a pandas dataframe. With fastai's `split_from_df()` method, we can split them later in a fastai Databunch. Just add a `is_valid` column (True or False) as an extra column. If is_valid[index] = True, then that example is put in the validation set and if is_valid[index] = False the example is put in the training set.

In [0]:
df_train_50k["is_valid"] = 'False'
df_train_50k.head()

,en,de,is_valid
0,"in china, people call it fazhan , or “development,” but in much of rest of the world, it is more commonly described simply as the “china boom,” or the “china miracle.”","in china sagt man dazu fazhan , also „entwicklung”, aber in den meisten teilen der welt wird dieses phänomen einfach als „china-boom“ oder „chinesisches wunder“ bezeichnet.",False
1,such pessimism is warranted.,ein derartiger pessimismus ist gerechtfertigt.,False
2,"that is also what, if you take her literally, merkel is doing now.","dasselbe tut derzeit auch merkel, wenn man sie beim wort nimmt.",False
3,"it proved to be the tipping point, triggering waves of protests by lawyers and other groups in pakistan’s main cities.","der vorfall erwies sich als wendepunkt, der wellen des protests von anwälten und anderen gruppen in pakistans größten städten auslöste.",False
4,"welfare-to-work programs in michigan and wisconsin served as the model for federal welfare reform under president bill clinton, and obamacare is based on massachusetts’ health-care system, introduced under republican governor mitt romney.","arbeitsbeschaffungsprogramme in michigan und wisconsin dienten als modell für die staatliche sozialreform unter präsident bill clinton, und das krankenversicherungsprogramm „obamacare“ baut auf dem gesundheitssystem von massachusetts auf, das unter dem republikanischen gouverneur mitt romney eingeführt wurde.",False


In [0]:
df_valid["is_valid"] = 'True'
df_valid.head()

,en,de,is_valid
0,a republican strategy to counter the re-election of obama,"eine republikanische strategie, um der wiederwahl von obama entgegenzutreten",True
1,republican leaders justified their policy by the need to combat electoral fraud.,"die führungskräfte der republikaner rechtfertigen ihre politik mit der notwendigkeit, den wahlbetrug zu bekämpfen.",True
2,"however, the brennan centre considers this a myth, stating that electoral fraud is rarer in the united states than the number of people killed by lightning.","allerdings hält das brennan center letzteres für einen mythos, indem es bekräftigt, dass der wahlbetrug in den usa seltener ist als die anzahl der vom blitzschlag getöteten menschen.",True
3,"indeed, republican lawyers identified only 300 cases of electoral fraud in the united states in a decade.",die rechtsanwälte der republikaner haben in 10 jahren in den usa übrigens nur 300 fälle von wahlbetrug verzeichnet.,True
4,one thing is certain: these new provisions will have a negative impact on voter turn-out.,eins ist sicher: diese neuen bestimmungen werden sich negativ auf die wahlbeteiligung auswirken.,True


In [0]:
df_train_valid = pd.concat([df_train_50k, df_valid]).reset_index(drop=True)
df_train_valid

,en,de,is_valid
0,"in china, people call it fazhan , or “development,” but in much of rest of the world, it is more commonly described simply as the “china boom,” or the “china miracle.”","in china sagt man dazu fazhan , also „entwicklung”, aber in den meisten teilen der welt wird dieses phänomen einfach als „china-boom“ oder „chinesisches wunder“ bezeichnet.",False
1,such pessimism is warranted.,ein derartiger pessimismus ist gerechtfertigt.,False
2,"that is also what, if you take her literally, merkel is doing now.","dasselbe tut derzeit auch merkel, wenn man sie beim wort nimmt.",False
3,"it proved to be the tipping point, triggering waves of protests by lawyers and other groups in pakistan’s main cities.","der vorfall erwies sich als wendepunkt, der wellen des protests von anwälten und anderen gruppen in pakistans größten städten auslöste.",False
4,"welfare-to-work programs in michigan and wisconsin served as the model for federal welfare reform under president bill clinton, and obamacare is based on massachusetts’ health-care system, introduced under republican governor mitt romney.","arbeitsbeschaffungsprogramme in michigan und wisconsin dienten als modell für die staatliche sozialreform unter präsident bill clinton, und das krankenversicherungsprogramm „obamacare“ baut auf dem gesundheitssystem von massachusetts auf, das unter dem republikanischen gouverneur mitt romney eingeführt wurde.",False
5,"the french have been the most insistent that europe has interests that are not identical to america’s – particularly in the middle east, where france has been pro-arab.","die franzosen haben am meisten darauf bestanden, dass es europäische interessen gibt, die nicht mit denen amerikas identisch sind – vor allem im nahen osten, wo frankreich bislang proarabisch war.",False
6,"in all of these areas, trump has reversed attractive american policies.",in allen diesen bereichen hat trump die attraktivität der amerikanischen politik verringert.,False
7,"so far, park, sensing that the electorate trusts her foreign-policy instincts, has kept her north korea cards close to her chest.","park hat erkannt, dass die wählerschaft ihren außenpolitischen instinkten vertraut, und bislang niemanden in ihre nordkorea-karten schauen lassen.",False
8,and ronald reagan launched his 1980 campaign near the mississippi town where three civil-rights activists were murdered by white supremacists in 1964.,"und ronald reagan hat seinen wahlkampf 1980 nahe der stadt in mississippi eröffnet, wo drei bürgerrechtsaktivisten 1964 von verfechtern der weißen vorherrschaft ermordet worden waren.",False
9,the governments that launched emu anticipated that their workers and businesses would eventually be sufficiently mobile and flexible to adjust to economic changes affecting them differentially.,"die an der ewu beteiligten regierungen gingen davon aus, dass arbeitskräfte und firmen letztlich ausreichend mobilität und flexibilität entwickeln würden, um sich den, sie in unterschiedlichen ausmaß betreffenden wirtschaftlichen veränderungen, anzupassen.",False


In [0]:
df_train_valid.to_csv(path_translation_data + 'news-commentary-v14-50k-train-valid.en-de.csv', index=False)

In [0]:
df_train_valid_50k = pd.read_csv(path_translation_data + 'news-commentary-v14-50k-train-valid.en-de.csv')

In [0]:
df_train_valid_50k.isnull().any()

en          False
de          False
is_valid    False
dtype: bool

## Necessary methods for creating a DataBunch for a Machine Translation model

In [0]:
def seq2seq_collate(samples, pad_idx=1, pad_first=True, backwards=False):
    "Function that collect samples and adds padding. Flips token order if needed"
    samples = to_data(samples) #https://docs.fast.ai/torch_core.html#to_data #to their int value?
    max_len_x,max_len_y = max([len(s[0]) for s in samples]),max([len(s[1]) for s in samples]) #max length of a sentence in the dataset
    res_x = torch.zeros(len(samples), max_len_x).long() + pad_idx #samples x max_len of whole data 
    res_y = torch.zeros(len(samples), max_len_y).long() + pad_idx
    if backwards: pad_first = not pad_first
    for i,s in enumerate(samples): #go through every sample
        if pad_first: #padding tokens first then emb
            res_x[i,-len(s[0]):],res_y[i,-len(s[1]):] = LongTensor(s[0]),LongTensor(s[1]) #row 0, 1, 2 (one sentence in each language each)
        else:         
            res_x[i,:len(s[0]):],res_y[i,:len(s[1]):] = LongTensor(s[0]),LongTensor(s[1])
    if backwards: res_x,res_y = res_x.flip(1),res_y.flip(1)
    return res_x,res_y

In [0]:
class Seq2SeqDataBunch(TextDataBunch):
    "Create a `TextDataBunch` suitable for training an RNN classifier."
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs:int=32, val_bs:int=None, pad_idx=1,
               dl_tfms=None, pad_first=False, device:torch.device=None, no_check:bool=False, backwards:bool=False, **dl_kwargs) -> DataBunch:
        "Function that transform the `datasets` in a `DataBunch` for classification. Passes `**dl_kwargs` on to `DataLoader()`"
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
        val_bs = ifnone(val_bs, bs)
        collate_fn = partial(seq2seq_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=backwards)
        train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs//2)
        train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **dl_kwargs)
        dataloaders = [train_dl]
        for ds in datasets[1:]:
            lengths = [len(t) for t in ds.x.items]
            sampler = SortSampler(ds.x, key=lengths.__getitem__)
            dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
        return cls(*dataloaders, path=path, device=device, collate_fn=collate_fn, no_check=no_check)

In [0]:
class Seq2SeqTextList(TextList):
    _bunch = Seq2SeqDataBunch
    _label_cls = TextList

## Option 1:

Use `newstest2013` as own validation set

https://docs.fast.ai/data_block.html#Step-1:-Provide-inputs

1.   Step 1: Provide inputs
2.   Step 2: Split the data between the training and the validation set
3.   Step 3: Label the inputs
4.   optional: add test set: https://docs.fast.ai/data_block.html#LabelLists.add_test_folder





**split_from_df()** looks up the `is_valid` column of our dataframe and separates training and validation set

In [0]:
src = (Seq2SeqTextList.from_df(df_train_valid_50k, path=path_translation_data, cols='en') #Step 1
       .split_from_df() #Step 2
       .label_from_df(cols='de', label_cls=TextList)) #Step 3

In [0]:
src

LabelLists;

Train: LabelList (50000 items)
x: Seq2SeqTextList
xxbos in china , people call it xxunk , or “ development , ” but in much of rest of the world , it is more commonly described simply as the “ china boom , ” or the “ china miracle . ”,xxbos such pessimism is warranted .,xxbos that is also what , if you take her literally , merkel is doing now .,xxbos it proved to be the tipping point , triggering waves of protests by lawyers and other groups in pakistan ’s main cities .,xxbos welfare - to - work programs in michigan and xxunk served as the model for federal welfare reform under president bill clinton , and obamacare is based on massachusetts ’ health - care system , introduced under republican governor mitt romney .
y: TextList
xxbos in china sagt man dazu xxunk , also „ entwicklung ” ,    aber in den meisten teilen der welt wird dieses phänomen einfach als „ china - boom “ oder „ chinesisches wunder “ bezeichnet .,xxbos ein derartiger pessimismus ist gerechtfertigt .,xxbos

Length (number of tokens) of a sentence for the 90th percentile (en=43.0, en=44.0)

In [0]:
np.percentile([len(o) for o in src.train.x.items] + [len(o) for o in src.valid.x.items], 90)

43.0

In [0]:
np.percentile([len(o) for o in src.train.y.items] + [len(o) for o in src.valid.y.items], 90)

44.0

For faster training and better accuracy (the longer a sentence the worse the generated translation tend to be), we remove the examples that have 40 tokens long. You can go lower than 40 of course (I do 25 in Option 2 below for example)

In [0]:
src = src.filter_by_func(lambda x,y: len(x) > 40 or len(y) > 40)

In [0]:
len(src.train), len(src.valid)

(41477, 2677)

In [0]:
data = src.databunch()

In [0]:
data.save("db1_41k")

## Option 2:

We create our validation set from our training set

In [0]:
df_train = pd.read_csv(path_translation_data + 'news-commentary-v14.en-de.csv')

In [0]:
df_train.isnull().any()

en    False
de    False
dtype: bool

In [0]:
df_train['en'] = df_train['en'].apply(lambda x:x.lower())
df_train['de'] = df_train['de'].apply(lambda x:x.lower())

In [0]:
len(df_train)

329167

In [0]:
pd.set_option('max_colwidth', 800)

In [0]:
df_train.tail()

,en,de
329162,their achievement remains one of the greatest in recent history.,das bleibt eine der größten errungenschaften in der jüngeren geschichte.
329163,"at the same time, zuma’s revolutionary generation still seems uneasy leading south africa in a post-apartheid era that is now 15 years old.",gleichzeitig scheint sich zumas revolutionäre generation mit der führung südafrikas in der nun seit 15 jahren dauernden ära nach der apartheid noch immer unwohl zu fühlen.
329164,"in a region that reveres the elderly, zuma’s attachment to his rural traditions must be matched by an equal openness to the appetites of the country’s youth.","in einer region, wo die älteren menschen sehr verehrt werden, muss zumas bindung an landestypische traditionen eine gleichwertige offenheit gegenüber den bedürfnissen der jugend des landes gegenüberstehen."
329165,"three in ten south africans are younger than 15, meaning that they did not live a day under apartheid.","drei von zehn südafrikanern sind jünger als 15 und das bedeutet, dass sie nicht einen tag unter der apartheid gelebt haben."
329166,"somehow zuma must find a way to honor his own generation’s commitment to racial justice and national liberation, while empowering the masses who daily suffer the sting of class differences and yearn for material gain.","irgendwie muss zuma einen weg finden, einerseits das engagement seiner generation hinsichtlich ethnischer gerechtigkeit und nationaler befreiung zu würdigen und andererseits den massen, die täglich unter klassenunterschieden leiden und sich nach materiellen verbesserungen sehnen, mehr mitwirkungsmöglichkeiten einzuräumen."


**split_from_rand_pct** takes a percentage(valid_pct = 0.2 (default)) from our training set as validation set.

In [0]:
src = (Seq2SeqTextList.from_df(df_train, path=path_translation_data, cols='en') #Step 1
       .split_by_rand_pct(valid_pct=0.1,seed=42) #Step 2
       .label_from_df(cols='de', label_cls=TextList)) #Step 3

In [0]:
src.train.x.items[:5]

array([array([   2,  157, 5307, 1546,  101]),
       array([   2, 4244, 6542,   24,   23,   32,  361,   62,  856,   13,   33,   16, 2614, 4883,   81,    9,  426,   12,
       1546,   11]),
       array([    2,   146,   160,  1290,    10,  3065,   737,  5196, 21369,    14, 26370, 26371,   355, 26372, 24364,    19,
           0, 14099,  6297, 14881,   136,  2466,    10, 13732,  4913,   116,  1546,    20,   454,    11]),
       array([  2,  48, 290, 322, 457,  23, 101]),
       array([   2,  169,   38, 5628, 2459,   10,    9,  317,   12, 1546,   32, 2026,  119,  184,  372,   11])],
      dtype=object)

In [0]:
len(src.train.x.items[0])

5

In [0]:
src.train.x[0]

Text xxbos $ 10,000 gold ?

In [0]:
np.percentile([len(o) for o in src.train.x.items] + [len(o) for o in src.valid.x.items], 90)

43.0

In [0]:
np.percentile([len(o) for o in src.train.y.items] + [len(o) for o in src.valid.y.items], 90)

44.0

For even more efficient training, let's just keep the sentences with less than < 25 tokens (in Option 1 we just removed sentences with 40 tokens which is probably still too long):

In [0]:
src_2 = src.filter_by_func(lambda x,y: len(x) > 25 or len(y) > 25)

In [0]:
len(src_2.train), len(src_2.valid)

(137400, 15335)

In [0]:
data = src_2.databunch()

In [0]:
data.show_batch()

text,target
"xxbos they are suffering huge financial losses by boycotting the courts , and yet they are determined that the chief justice must be reinstated .","xxbos durch ihren xxunk erleiden sie xxunk finanzielle verluste und dennoch halten sie unbeirrt daran fest , dass der höchstrichter wieder eingesetzt werden muss ."
"xxbos of the 14 indicators of progress associated with the primary gender equity goal , sdg 5 , most countries are measuring just three .","xxbos von den 14 xxunk , die dem primärziel der gleichberechtigung , sdg 5 , zugeordnet sind , messen die meisten länder lediglich 3 ."
"xxbos germany , in particular , feels that it has no reason to share its imf representation with other , fiscally weaker eurozone members .","xxbos deutschland insbesondere sieht keinen anlass , seine iwf - vertretung mit anderen , fiskalisch schwächeren mitgliedern der eurozone zu teilen ."
"xxbos needless to say , the ability to comprehend change has become indispensable at a time when the world is experiencing tectonic geopolitical shifts .","xxbos die fähigkeit , wandel zu erkennen ist natürlich umso unverzichtbarer in einer zeit , in der die welt tektonische geopolitische veränderungen erlebt ."
"xxbos and yet m5s probably can not achieve a parliamentary majority , and it has vowed not to enter into coalitions with other parties .","xxbos dennoch wird die m5s wahrscheinlich keine parlamentsmehrheit erringen und sie versprach auch , nicht in koalitionen mit anderen parteien einzutreten ."


Vocabulary size for English (33272 tokens) and German (60000 tokens).

In [0]:
len(data.x.vocab.itos), len(data.y.vocab.itos)

(33272, 60000)

**Save the Databunch with 137400 training sentences and  15335 validation sentences which have less than 25 tokens each. We will use this Databunch for our further experiments.**

In [0]:
data.save("db1_137k_25sl")

## Save and Load Databunch (137k training size with maximum sentence length 25 tokens)

In [0]:
data = load_data(path_translation_data, "db1_137k_25sl")

In [0]:
data.show_batch()

text,target
"xxbos for example , better attribution forensics may enhance the role of punishment ; and better defenses through encryption may increase deterrence by denial .","xxbos so könnte etwa eine bessere xxunk die rolle der bestrafung verstärken , und bessere verteidigungsmechanismen durch verschlüsselung könnten die abschreckung durch verhinderung steigern ."
"xxbos hamas ’s capture of the gaza strip has created , along with iran , a second radical islamist state in the middle east .",xxbos die machtübernahme i m gazastreifen durch die hamas hat zusätzlich zum iran einen zweiten radikal - islamistischen staat i m nahen osten geschaffen .
xxbos the basel iii agreement on capital adequacy and other recent reforms still have not ring - fenced trade financing from these potential shocks .,xxbos das basel - iii - übereinkommen zur eigenkapitalausstattung und andere aktuelle reformen haben die handelsfinanzierung bisher noch nicht gegen diese potenziellen erschütterungen isoliert .
xxbos anyone who portrays the us as a huge loser from the global economic status quo needs to gain some perspective on the matter .,"xxbos wer die usa als großen verlierer des weltwirtschaftlichen status quo darstellt , muss seine sicht diesbezüglich dringend relativieren ."
"xxbos whereas labor - force participation in the eurozone is on the rise , it has been declining in the us since around 2000 .","xxbos während die erwerbsbeteiligung in der eurozone steigt , sinkt sie in den usa seit etwa dem jahr 2000 ."


In [0]:
data.train_ds[50]

(Text xxbos few people can muster great enthusiasm for the un .,
 Text xxbos wenigen menschen gelingt es , große begeisterung für die uno aufzubringen .)

In [0]:
data.train_ds[50][0].data[:]  #embedding for: "xxbos few people can muster great enthusiasm for the un ."

array([   2,  276,   84,   49, 8943,  273, 4374,   21,    9,  409,   11])

# Create Pretrained embeddings

Download the word embeddings (crawl vectors) from the fastText docs. FastText has [pre-trained word vectors](https://fasttext.cc/docs/en/crawl-vectors.html) for 157 languages, trained on Common Crawl and Wikipedia. These models were trained using CBOW. Load them with FastTexts `load_model()` method.

In [0]:
en_vecs = ft.load_model(str(path_translation_data + 'cc.en.300.bin'))

In [0]:
de_vecs = ft.load_model(path_translation_data + 'cc.de.300.bin')

Embedding module with the pretrained vectors and random data for the missing parts.

In [0]:
def create_emb(vecs, itos, em_sz=300, mult=1.): #parameters(downloaded vectors, vocabulary, embedding size, mult)
    emb = nn.Embedding(len(itos), em_sz, padding_idx=1)  #PyTorch Embedding
    wgts = emb.weight.data
    vec_dic = {w:vecs.get_word_vector(w) for w in vecs.get_words()} #create dict; word: vector; 1) get_word_vector: Get the vector representation of word.; 2) get_words(): Get the entire list of words of the dictionary
    miss = []
    for i,w in enumerate(itos):
        try: wgts[i] = tensor(vec_dic[w]) #save the vocabulary in wgts
        except: miss.append(w)
    return emb

In [0]:
emb_enc = create_emb(en_vecs, data.x.vocab.itos)
emb_dec = create_emb(de_vecs, data.y.vocab.itos)

In [0]:
emb_enc.weight.size(), emb_dec.weight.size() #English, German embedding matrix

(torch.Size([33272, 300]), torch.Size([60000, 300]))

In [0]:
emb_enc.num_embeddings, emb_enc.embedding_dim

(33272, 300)

In [0]:
emb_dec.num_embeddings, emb_dec.embedding_dim

(60000, 300)

In [0]:
torch.save(emb_enc, path_translation + 'en_emb_137k_25l.pth')

In [0]:
torch.save(emb_dec, path_translation + 'de_emb_137k_25l.pth')

In [0]:
emb_enc = torch.load(path_translation + 'en_emb_137k_25l.pth')
emb_dec = torch.load(path_translation + 'de_emb_137k_25l.pth')

In [0]:
emb_enc, type(emb_enc)

(Embedding(33272, 300, padding_idx=1), torch.nn.modules.sparse.Embedding)

In [0]:
emb_enc.weight.data ,emb_enc.weight.data.shape #English

(tensor([[ 0.6550,  0.9002, -0.6118,  ...,  1.4594,  0.5450, -0.5295],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.4711,  0.4144,  0.6080,  ...,  0.4429,  0.5999, -0.6418],
         ...,
         [-0.0192, -0.0574, -0.0462,  ...,  0.0495, -0.1114, -0.1833],
         [-0.0357, -0.0035, -0.1139,  ...,  0.0867, -0.0637, -0.0681],
         [-0.6883, -0.2589,  2.5070,  ...,  0.5557,  0.7431,  0.4535]]),
 torch.Size([33272, 300]))

In [0]:
emb_dec.weight.data ,emb_dec.weight.data.shape #German

(tensor([[-1.6373e+00,  9.0775e-01, -6.2051e-01,  ...,  1.5115e+00,
          -3.0657e-01,  4.7299e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.3741e-01, -4.2239e-01, -4.0976e-01,  ...,  9.2944e-01,
           1.1001e+00, -5.6154e-02],
         ...,
         [ 1.1834e-03, -2.0987e-02,  1.0963e-02,  ..., -1.4762e-02,
          -2.2906e-02, -3.2849e-03],
         [ 2.0987e-02, -7.4525e-01, -1.2339e+00,  ..., -5.1861e-02,
          -1.2668e+00, -8.9546e-01],
         [ 1.1385e+00,  9.1379e-02,  1.3485e+00,  ...,  4.9208e-02,
          -2.2599e+00,  8.0766e-01]]), torch.Size([60000, 300]))